# Data Wrangling with DataFrames.jl
Data wrangling is a core part of any data science project, and it makes life a lot easier if we can quickly and easily manipulate tabular data. The `DataFrames.jl` package provides the `DataFrame` type to hold tabular data, and numerous functions to operate on the data. It is also tightly integrated with other data science packages in Julia, such as `gadfly.jl` for visualization, and `GLM.jl` for statistical modelling.

# Getting Started
## Installing the DataFrames.jl and CSV.jl Packages
Before we can use the `DataFrames.jl` package, we need to install it. We will also install the `CSV.jl` package to read CSV files from disk, the `DataFramesMeta.jl` package to provide some more intuitive functions for working with DataFrames, and the `Pipe.jl` package to string multiple commands together in one call. To install these packages, follow these steps:

1. Open the Julia REPL
2. Type `]` to enter the package manager
3. Type `add DataFrames` to install the DataFrames package
4. Type `add CSV` to install the CSV package
5. Type `add DataFramesMeta` to install the DataFramesMeta package
6. Type `add Pipe` to install the Pipe package
7. Press CTRL-C to exit the package manager

## Loading Packages

Once you have installed these packages, load them into this notebook environment through the following commands:

In [1]:
using DataFrames
using CSV
using DataFramesMeta
using Pipe

## Downloading Data
For this tutorial, we will be using the COVID-19 dataset from Our World in Data, which contains data on case rates, death rates and vaccination rates for almost all countries across the course of the COVID-19 pandemic:
1. Navigate to the Our World in Data COVID-19 page [here](https://github.com/owid/covid-19-data/tree/master/public/data)
2. Scroll down to the start of the README (underneath the list of folders and files in the repo)
3. Click on the `CSV` link next to the title `Download our complete COVID-19 dataset`
4. Once the data has downloaded, move it into the same directory as this notebook.

# Creating DataFrames
There are two ways to create DataFrames:
1. Create a DataFrame from scratch using existing vectors
2. Load data from disk into a DataFrame

## Creating DataFrames From Scratch
DataFrames can be created from scratch, by first assigning values to vectors, and then constructing a DataFrame that holds all of these vectors as columns. If we want the same value to be repeated for all rows in a column, we can just pass one value when creating the DataFrame, and it will be repeated the correct number of times to fill all rows:

In [2]:
fruit_names = ["Apple", "Banana", "Clementine", "Damson", "Elderberry"]
fruit_numbers = [200, 175, 120, 50, 44]
df = DataFrame(
    fruit = fruit_names,
    stock = convert.(Float64, fruit_numbers),
    on_offer = false
)

,fruit,stock,on_offer
,String,Float64,Bool
1,Apple,200.0,0
2,Banana,175.0,0
3,Clementine,120.0,0
4,Damson,50.0,0
5,Elderberry,44.0,0


Note that as the DataFrame is created, the type of each column is inferred from the values. If you want to specify a type for a column (e.g. `Float64` instead of `Int64`), you need to broadcast the `convert()` function across all elements of the vector within the call to the DataFrame constructor.

## Creating DataFrames Row-by-Row
We can also create a empty DataFrame and fill it row-by-row. This makes use of the `push!()` function which changes the DataFrame in-place. The new row is specified as a tuple, where the elements are in the same order as the columns in the DataFrame:

In [24]:
df = DataFrame(Input=Int[], Squared=Int[])
inputs = 1:4
for i in inputs
    push!(df, (i, i^2))
end
println(df)

4×2 DataFrame
 Row │ Input  Squared 
     │ Int64  Int64   
─────┼────────────────
   1 │     1        1
   2 │     2        4
   3 │     3        9
   4 │     4       16


We can also pass a dict to `push!()`, where the keys match the column names of the DataFrame. This means that we don't have to worry about the order of the columns in the DataFrame that we are adding to:

In [29]:
df = DataFrame(Input=Int[], Squared=Int[])
inputs = 1:4
for i in inputs
    push!(df, Dict("Squared"=>i^2, "Input"=>i)) # note that column order is reversed, but this is handled
end
println(df)

4×2 DataFrame
 Row │ Input  Squared 
     │ Int64  Int64   
─────┼────────────────
   1 │     1        1
   2 │     2        4
   3 │     3        9
   4 │     4       16


**NOTE**: constructing a DataFrame row-by-row is much less efficient than constructing it all at once, so bear this in mind when working with large DataFrames.

## Creating DataFrames From A CSV File
We can also create a DataFrame from a CSV file, by using the `CSV` package and specifying `DataFrame` as the sink (the second argument):

In [2]:
covid = CSV.read("owid-covid-data.csv", DataFrame)
println("DataFrame created for COVID dataset")

DataFrame created for COVID dataset


# Extracting DataFrame Metadata

## DataFrame Dimensions
To get the number of rows and columns in a DataFrame, we can use the `size()` function:

In [61]:
size(covid)

(208111, 67)

## DataFrame Column Names
To get the name of each column in a DataFrame, we can use the `names()` function

In [6]:
names(covid)

67-element Vector{String}:
 "iso_code"
 "continent"
 "location"
 "date"
 "total_cases"
 "new_cases"
 "new_cases_smoothed"
 "total_deaths"
 "new_deaths"
 "new_deaths_smoothed"
 "total_cases_per_million"
 "new_cases_per_million"
 "new_cases_smoothed_per_million"
 ⋮
 "cardiovasc_death_rate"
 "diabetes_prevalence"
 "female_smokers"
 "male_smokers"
 "handwashing_facilities"
 "hospital_beds_per_thousand"
 "life_expectancy"
 "human_development_index"
 "excess_mortality_cumulative_absolute"
 "excess_mortality_cumulative"
 "excess_mortality"
 "excess_mortality_cumulative_per_million"

We can also filter the column names that are returned by `names()` based on their element type, or use a regex to match on the column name:

In [11]:
println(names(covid, String))
println(names(covid, r"cases"))

["location"]
["total_cases", "new_cases", "new_cases_smoothed", "total_cases_per_million", "new_cases_per_million", "new_cases_smoothed_per_million"]


Finally, we can use the `Not()` operator to exclude column names from being returned:

In [14]:
names(covid, Not(:iso_code))

66-element Vector{String}:
 "continent"
 "location"
 "date"
 "total_cases"
 "new_cases"
 "new_cases_smoothed"
 "total_deaths"
 "new_deaths"
 "new_deaths_smoothed"
 "total_cases_per_million"
 "new_cases_per_million"
 "new_cases_smoothed_per_million"
 "total_deaths_per_million"
 ⋮
 "cardiovasc_death_rate"
 "diabetes_prevalence"
 "female_smokers"
 "male_smokers"
 "handwashing_facilities"
 "hospital_beds_per_thousand"
 "life_expectancy"
 "human_development_index"
 "excess_mortality_cumulative_absolute"
 "excess_mortality_cumulative"
 "excess_mortality"
 "excess_mortality_cumulative_per_million"

## Viewing the Top and Bottom
We can view the top of a DataFrame using the `first()` function. This is especially useful for inspecting data after we import it, to get an idea of what the columns contain. By default this only shows us the first entry, but we can specify the number of entries that we want to view. For large DataFrames, only the first few columns will be shown, and a message will be printed to say how many columns have been left out:

In [40]:
first(covid, 5)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed
,String15,String15?,String,Date,Float64?,Float64?,Float64?
1,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,missing
2,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,missing
3,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,missing
4,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,missing
5,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,missing


Similarly, we can use the `last()` function to view the bottom of the DataFrame, and specify the number of rows:

In [37]:
last(covid, 3)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed
,String15,String15?,String,Date,Float64?,Float64?,Float64?
1,ZWE,Africa,Zimbabwe,2022-08-09,256490.0,3.0,12.429
2,ZWE,Africa,Zimbabwe,2022-08-10,256492.0,2.0,9.857
3,ZWE,Africa,Zimbabwe,2022-08-11,256513.0,21.0,9.857


If we want to view all of the rows and/or columns, we can use the `show()` function and set the `allrows` or `allcols` arguments to `true`:

In [42]:
show(covid, allcols=true)

208111×67 DataFrame
    Row │ iso_code  continent  location     date        total_cases  new_cases  new_cases_smoothed  total_deaths  new_deaths  new_deaths_smoothed  total_cases_per_million  new_cases_per_million  new_cases_smoothed_per_million  total_deaths_per_million  new_deaths_per_million  new_deaths_smoothed_per_million  reproduction_rate  icu_patients  icu_patients_per_million  hosp_patients  hosp_patients_per_million  weekly_icu_admissions  weekly_icu_admissions_per_million  weekly_hosp_admissions  weekly_hosp_admissions_per_million  total_tests  new_tests  total_tests_per_thousand  new_tests_per_thousand  new_tests_smoothed  new_tests_smoothed_per_thousand  positive_rate  tests_per_case  tests_units  total_vaccinations  people_vaccinated  people_fully_vaccinated  total_boosters   new_vaccinations  new_vaccinations_smoothed  total_vaccinations_per_hundred  people_vaccinated_per_hundred  people_fully_vaccinated_per_hundred  total_boosters_per_hundred  new_vaccinations_smoothed_

# Subsetting DataFrames

## Subsetting Rows and Columns by Position
We can take a subset of the rows and columns by specifying the index range in square brackets after the DataFrame name. There are two important things to note about indexing by row/column number:
1. The range is 1-based
2. The range is inclusive i.e. 1:5 returns the first to the fifth

In [49]:
covid[1:5, 1:5]

,iso_code,continent,location,date,total_cases
,String15,String15?,String,Date,Float64?
1,AFG,Asia,Afghanistan,2020-02-24,5.0
2,AFG,Asia,Afghanistan,2020-02-25,5.0
3,AFG,Asia,Afghanistan,2020-02-26,5.0
4,AFG,Asia,Afghanistan,2020-02-27,5.0
5,AFG,Asia,Afghanistan,2020-02-28,5.0


We can also select columns by name, and combine this with row number indexing. We need to prefix each column name with a `:` to let Julia know that we are referring to a column name within the DataFrame:

In [53]:
covid[1:5, [:iso_code, :total_cases]]

,iso_code,total_cases
,String15,Float64?
1,AFG,5.0
2,AFG,5.0
3,AFG,5.0
4,AFG,5.0
5,AFG,5.0


## Filtering Rows by Value
To filter rows based on their values rather than positions, we can use the `subset()` function (or `subset!()` to act in-place). This function takes the dataframe as its first argument, and the conditions as the subsequent arguments. Each condition is specified in 2 parts, separated by a `->`:
1. The column name to filter on (preceded by a `:`) and the name to temporarily assign it to, separated by a `=>`. Note that this temporary name will **not** be used in the filtered DataFrame
2. The condition that each row needs to satisfy

In [15]:
println("Before filtering:")
println(df)
println("After filtering:")
print(subset(df, :stock => Stock -> Stock .> 100))

Before filtering:
5×3 DataFrame
 Row │ fruit       stock    on_offer 
     │ String      Float64  Bool     
─────┼───────────────────────────────
   1 │ Apple         200.0     false
   2 │ Banana        175.0     false
   3 │ Clementine    120.0     false
   4 │ Damson         50.0     false
   5 │ Elderberry     44.0     false
After filtering:
3×3 DataFrame
 Row │ fruit       stock    on_offer 
     │ String      Float64  Bool     
─────┼───────────────────────────────
   1 │ Apple         200.0     false
   2 │ Banana        175.0     false
   3 │ Clementine    120.0     false

The syntax of the `subset()` function from `DataFrames.jl` is quite long-winded. Luckily, the `DataFramesMeta.jl` package provides an equivalent `@subset()` function which does the same thing, but with much more intuitive syntax

In [37]:
println("After filtering with DataFramesMeta.jl:")
print(@subset(df, :stock .> 100))

After filtering with DataFramesMeta.jl:
3×4 DataFrame
 Row │ fruit       stock    on_offer  yellow 
     │ String      Float64  Bool      Bool   
─────┼───────────────────────────────────────
   1 │ Apple         200.0     false   false
   2 │ Banana        175.0     false    true
   3 │ Clementine    120.0     false   false

## Filtering Columns by Value
Just as `subset()` allows us to select rows by their value, `transform()` allows us to select columns by their name. In the most simple case, we can select a single column:

In [16]:
select(df, :fruit)

,fruit
,String
1,Apple
2,Banana
3,Clementine
4,Damson
5,Elderberry


If we want more than one column, we **don't** need to put the column names in a vector:

In [32]:
select(df, :fruit, :stock)

,fruit,stock
,String,Float64
1,Apple,200.0
2,Banana,175.0
3,Clementine,120.0
4,Damson,50.0
5,Elderberry,44.0


We can also use regular expressions to select all columns matching a certain pattern:

In [20]:
select(df, r"t")

,fruit,stock
,String,Float64
1,Apple,200.0
2,Banana,175.0
3,Clementine,120.0
4,Damson,50.0
5,Elderberry,44.0


We can take all columns **apart** from certain ones by using the `Not()` operator in combination with any of the operators outlined above:

In [22]:
select(df, Not(r"t"))

,on_offer
,Bool
1,0
2,0
3,0
4,0
5,0


# Extracting DataFrame Values
Once we have selected the rows and columns that we are interested in, we often want to extract the data held in a particular row or column e.g. to use the values in another operation.

## Extracting Column Values
We can extract the values in a column as a vector using two types of notation:
1. Specifying the name of the column with the `.` operator
2. Using subset indexing (which also allows us to specify the number of rows to select values from)

In [15]:
covid.iso_code

208111-element PooledArrays.PooledVector{String15, UInt32, Vector{UInt32}}:
 "AFG"
 "AFG"
 "AFG"
 "AFG"
 "AFG"
 "AFG"
 "AFG"
 "AFG"
 "AFG"
 "AFG"
 "AFG"
 "AFG"
 "AFG"
 ⋮
 "ZWE"
 "ZWE"
 "ZWE"
 "ZWE"
 "ZWE"
 "ZWE"
 "ZWE"
 "ZWE"
 "ZWE"
 "ZWE"
 "ZWE"
 "ZWE"

In [16]:
covid[1:5, :iso_code]

5-element PooledArrays.PooledVector{String15, UInt32, Vector{UInt32}}:
 "AFG"
 "AFG"
 "AFG"
 "AFG"
 "AFG"

Note that with the indexing syntax above, the column values are returned as a vector. If we want them as a one-column DataFrame instead, we wrap the column index in square brackets:

In [18]:
covid[1:5, [:iso_code]]

,iso_code
,String15
1,AFG
2,AFG
3,AFG
4,AFG
5,AFG


## Extracting Row Values
Julia doesn't seem to have a built-in function to extract the values in each column of a particular row to a dict, but we can define our own function:

In [58]:
function extract_df_row(df, row_index)
    row_contents = Dict()
    for i in enumerate(names(df))
        row_contents[i[2]] = df[1,i[1]]
    end
    row_contents
end

println(df[1,:])

println(extract_df_row(df, 1))

DataFrameRow
 Row │ fruit   stock    on_offer  yellow 
     │ String  Float64  Bool      Bool   
─────┼───────────────────────────────────
   1 │ Apple     200.0     false   false
Dict{Any, Any}("fruit" => "Apple", "yellow" => false, "stock" => 200.0, "on_offer" => false)


# Adding DataFrame Columns
We can add new columns to a dataframe in a number of ways, either using new data or by transforming the existing values.

## Adding a Vector as a New Column
If we have a vector with as many elements as the dataframe has rows, we can add it using the `dataframe.new_column` syntax that we saw for selecting values in an existing column. Note that this changes the dataframe in-place:

In [28]:
println("Before adding new column =")
println(df)
df.yellow = [false, true, false, false, false]
println("After adding new column =")
println(df)

Before adding new column =
5×3 DataFrame
 Row │ fruit       stock    on_offer 
     │ String      Float64  Bool     
─────┼───────────────────────────────
   1 │ Apple         200.0     false
   2 │ Banana        175.0     false
   3 │ Clementine    120.0     false
   4 │ Damson         50.0     false
   5 │ Elderberry     44.0     false
After adding new column =
5×4 DataFrame
 Row │ fruit       stock    on_offer  yellow 
     │ String      Float64  Bool      Bool   
─────┼───────────────────────────────────────
   1 │ Apple         200.0     false   false
   2 │ Banana        175.0     false    true
   3 │ Clementine    120.0     false   false
   4 │ Damson         50.0     false   false
   5 │ Elderberry     44.0     false   false


## Creating New Columns from Existing Columns
We can use the `select()` method to make new columns from existing ones. In the most simple case, we can select a column and copy its values into a new column with the `:existing_column => new_column` syntax:

In [31]:
select(df, :fruit, :fruit => :item)

,fruit,item
,String,String
1,Apple,Apple
2,Banana,Banana
3,Clementine,Clementine
4,Damson,Damson
5,Elderberry,Elderberry


We can also transform the values of an existing column and add the resulting values to a new column, by inserting some operations after the first `=>` operator. Note that the results of these operations need to be assigned to the new column with a second `=>` operator:

In [35]:
select(df, :stock, :stock => (x -> x .+ 1) => :stock_plus_one)

,stock,stock_plus_one
,Float64,Float64
1,200.0,201.0
2,175.0,176.0
3,120.0,121.0
4,50.0,51.0
5,44.0,45.0


We can do the same thing with an easier syntax with the `@select()` function from the `DataFramesMeta.jl` package:

In [43]:
@select(df, :stock, :stock_plus_one = :stock .+ 1)

,stock,stock_plus_one
,Float64,Float64
1,200.0,201.0
2,175.0,176.0
3,120.0,121.0
4,50.0,51.0
5,44.0,45.0


### Changing the Type of a Column
To change a column of integers or floats to a string, we can broadcast the `string()` function across all values of the column, and assign the result as a new column

In [5]:
df[!,:stock_string] = string.(df[!,:stock])
df

,fruit,stock,on_offer,stock_string
,String,Float64,Bool,String
1,Apple,200.0,0,200.0
2,Banana,175.0,0,175.0
3,Clementine,120.0,0,120.0
4,Damson,50.0,0,50.0
5,Elderberry,44.0,0,44.0


# Split-Apply-Combine
Many of the most common jobs in data munging follow the split-apply-combine paradigm:
1. Split the data into chunks, where the rows in each chunk share a value of a certain column
2. Apply an arithmetic operation to each chunk e.g. find the minimum value
3. Combine the results of the operation performed on each chunk into one dataframe

## Split
We can split a dataset into chunks using the `groupby()` function. This returns a `GroupedDataFrame` object:

In [60]:
groupby(covid, :continent)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed
,String15,String15?,String,Date,Float64?,Float64?,Float64?
1,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,missing
2,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,missing
3,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,missing
4,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,missing
5,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,missing
6,AFG,Asia,Afghanistan,2020-02-29,5.0,0.0,0.714
7,AFG,Asia,Afghanistan,2020-03-01,5.0,0.0,0.714
8,AFG,Asia,Afghanistan,2020-03-02,5.0,0.0,0.0
9,AFG,Asia,Afghanistan,2020-03-03,5.0,0.0,0.0


We can group by as many columns as we want, but we must pass the columns as a list:

In [62]:
groupby(covid, [:continent, :location])

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed
,String15,String15?,String,Date,Float64?,Float64?,Float64?
1,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,missing
2,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,missing
3,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,missing
4,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,missing
5,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,missing
6,AFG,Asia,Afghanistan,2020-02-29,5.0,0.0,0.714
7,AFG,Asia,Afghanistan,2020-03-01,5.0,0.0,0.714
8,AFG,Asia,Afghanistan,2020-03-02,5.0,0.0,0.0
9,AFG,Asia,Afghanistan,2020-03-03,5.0,0.0,0.0


## Apply & Combine
Once we have a `GroupedDataFrame` object, we can perform operations on each group. We can do this with the `@combine()` macro from `DataFramesMeta.jl`, where the first argument is the name of the `GroupedDataFrame`, and the subsequent arguments are operation specifications. Each operation is specified with the new column name first, then the `=` operator, then the function to be applied to the values in a specific column. When applying these operations, we can skip missing values in each group by wrapping the column in a `skipmissing()` statement:

In [77]:
covid_continents = groupby(covid, :continent)
@combine(covid_continents, :highest_daily_cases = maximum(skipmissing(:new_cases)))

,continent,highest_daily_cases
,String15?,Float64
1,Asia,621317.0
2,missing,4.07921e6
3,Europe,527487.0
4,Africa,41576.0
5,North America,1.38391e6
6,South America,287149.0
7,Oceania,175271.0


# Joining DataFrames
We can do the same range of joins with Julia `DataFrames` as we can in Python and R (inner, outer, left, right etc). For each function, we specify the column to use for joining with the `on` argument

In [82]:
people = DataFrame(ID=[20, 40, 60], Name=["John Doe", "Jane Doe", "Julia Doe"])
jobs = DataFrame(ID=[20, 40, 80], Job=["Lawyer", "Doctor", "Zoologist"])

,ID,Job
,Int64,String
1,20,Lawyer
2,40,Doctor
3,80,Zoologist


In [83]:
innerjoin(people, jobs, on=:ID)

,ID,Name,Job
,Int64,String,String
1,20,John Doe,Lawyer
2,40,Jane Doe,Doctor


In [84]:
outerjoin(people, jobs, on=:ID)

,ID,Name,Job
,Int64,String?,String?
1,20,John Doe,Lawyer
2,40,Jane Doe,Doctor
3,60,Julia Doe,missing
4,80,missing,Zoologist


In [85]:
leftjoin(people, jobs, on=:ID)

,ID,Name,Job
,Int64,String,String?
1,20,John Doe,Lawyer
2,40,Jane Doe,Doctor
3,60,Julia Doe,missing


In [86]:
rightjoin(people, jobs, on=:ID)

,ID,Name,Job
,Int64,String?,String
1,20,John Doe,Lawyer
2,40,Jane Doe,Doctor
3,80,missing,Zoologist


# Sorting DataFrames
The `DataFramesMeta.jl` package provides the `@orderby()` macro to sort a DataFrame by the values in a specific column

In [88]:
println("Before sorting =")
println(df)
println("After sorting =")
println(@orderby(df, :stock))

Before sorting =
5×4 DataFrame
 Row │ fruit       stock    on_offer  yellow 
     │ String      Float64  Bool      Bool   
─────┼───────────────────────────────────────
   1 │ Apple         200.0     false   false
   2 │ Banana        175.0     false    true
   3 │ Clementine    120.0     false   false
   4 │ Damson         50.0     false   false
   5 │ Elderberry     44.0     false   false
After sorting =
5×4 DataFrame
 Row │ fruit       stock    on_offer  yellow 
     │ String      Float64  Bool      Bool   
─────┼───────────────────────────────────────
   1 │ Elderberry     44.0     false   false
   2 │ Damson         50.0     false   false
   3 │ Clementine    120.0     false   false
   4 │ Banana        175.0     false    true
   5 │ Apple         200.0     false   false


By default, numeric values are sorted in descending order, but we can reverse this ordering by putting the `-` operator before the column name

In [90]:
println("After sorting by stock in descending order =")
println(@orderby(df, -:stock))

After sorting by descending order =
5×4 DataFrame
 Row │ fruit       stock    on_offer  yellow 
     │ String      Float64  Bool      Bool   
─────┼───────────────────────────────────────
   1 │ Apple         200.0     false   false
   2 │ Banana        175.0     false    true
   3 │ Clementine    120.0     false   false
   4 │ Damson         50.0     false   false
   5 │ Elderberry     44.0     false   false


# Piping DataFrame Commands
Just as dplyr in R has the `%>%` operator to pipe the output of one command to the next, so the `Pipe` package provides us with the `|>` operator. Note that in contrast to dplyr, the output of the last command is not silently passed to the next command; instead, the output of the operation to the left of `|>` must be explicitly referenced as `_` in the next function:

In [104]:
@pipe df |>
    select(_, :fruit, :stock, :yellow) |>
    groupby(_, :yellow) |>
    @combine(_, :highest_stock = maximum(:stock)) |>
    @orderby(_, :highest_stock)

,yellow,highest_stock
,Bool,Float64
1,1,175.0
2,0,200.0
